In [4]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [5]:
from scipy.stats import pearsonr
import shutil

In [6]:
T=pd.read_excel('../raw_data/climate_change_download_0.xls',index_col=1,na_values='..')
T.columns=T.columns.astype(str)
T=T.loc[T['Series name']=='Average daily min/max temperature (1961-1990, Celsius)','2011']

T=T.dropna().map(lambda s: s.split('/ ')[1]).astype(float)
T.name='Temperature'
T.rename(index={'Iran, Islamic Rep.':'Iran', 
                'Russian Federation':'Russia',
               'Korea, Dem. Rep.':'South Korea',
               'Hong Kong SAR, China':'Hong Kong',
               'Slovak Republic':'Slovakia',
               'Venezuela, RB':'Venezuela'},inplace=True)

In [10]:
H= pd.read_excel('../raw_data/Hip_fracture.xlsx',index_col=0)

In [11]:


Latitude= pd.read_excel('../raw_data/Latitudes.xlsx',index_col=0)
Latitude['latitude']=Latitude.Latitude.str[:2].astype(int)
Latitude.loc[(Latitude.Latitude.str[-1]=='S'),'latitude']*=-1

D= H.join(T)
D['Latitude']= Latitude.latitude.abs()

In [12]:
D.index.difference(T.index)

Index(['Serbia', 'Taiwan'], dtype='object')

In [13]:
vitD= pd.read_table('../data/VitaminD_per_country.tsv',index_col=0,squeeze=True)
vitD.rename(index={'USA':'United States','Korea (South)':'South Korea'},inplace=True)
D['VitaminD']=vitD

vitD.index.difference(D.index)

Index(['Cameroon', 'Estonia', 'Fidji Islands', 'Gambia', 'Vietnam'], dtype='object')

# Calcium

In [16]:
C= pd.read_excel('../raw_data/Calcium.xlsx')

In [17]:
C.index= C.Country.map(lambda s: s.split('[')[0].strip())
assert ~C.index.duplicated().any()

In [18]:
C.index[C.index.str.contains('Kong')]

Index([], dtype='object', name='Country')

In [19]:
C.index.difference(D.index)

Index(['Algeria', 'Bangladesh', 'Barbados', 'Bolivia', 'Botswana',
       'Burkina Faso', 'Cameroon', 'Cape Verde', 'Costa Rica', 'Egypt',
       'Ethiopia', 'Gambia', 'Guam', 'Jamaica', 'Latvia', 'Mali', 'Myanmar',
       'Nepal', 'Pakistan', 'Palestine', 'Polandi', 'Tanzania', 'Tonga', 'UK',
       'USA', 'Uganda', 'Vietnam'],
      dtype='object', name='Country')

In [20]:
D.index.difference(C.index)

Index(['Hong Kong', 'Lebanon', 'Lithuania', 'Malta', 'Oman', 'Poland',
       'Romania', 'Saudi Arabia', 'Slovakia', 'Slovenia', 'Tunisia', 'Turkey',
       'United Kingdom', 'United States', 'Venezuela'],
      dtype='object', name='Country')

In [22]:
D['Calcium']= C.iloc[:,0]

In [24]:
D.to_excel('../data/Data_for_correlations.xlsx')
